# Recognize named entities on Twitter with LSTMs

In this assignment, you will use a recurrent neural network to solve Named Entity Recognition (NER) problem. NER is a common task in natural language processing systems. It serves for extraction such entities from the text as persons, organizations, locations, etc. In this task you will experiment to recognize named entities from Twitter.

For example, we want to extract persons' and organizations' names from the text. Than for the input text:

    Ian Goodfellow works for Google Brain

a NER model needs to provide the following sequence of tags:

    B-PER I-PER    O     O   B-ORG  I-ORG

Where *B-* and *I-* prefixes stand for the beginning and inside of the entity, while *O* stands for out of tag or no tag. Markup with the prefix scheme is called *BIO markup*. This markup is introduced for distinguishing of consequent entities with similar types.

A solution of the task will be based on neural networks, particularly, on Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Libraries

For this task you will need the following libraries:
 - [Tensorflow](https://www.tensorflow.org) — an open-source software library for Machine Intelligence.
 
In this assignment, we use Tensorflow 1.15.0. You can install it with pip:

    !pip install tensorflow==1.15.0
     
 - [Numpy](http://www.numpy.org) — a package for scientific computing.
 
If you have never worked with Tensorflow, you would probably need to read some tutorials during your work on this assignment, e.g. [this one](https://www.tensorflow.org/tutorials/recurrent) could be a good starting point. 

### Data

The following cell will download all data required for this assignment into the folder `week2/data`.

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week2()

import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

File data/train.txt is already downloaded.
File data/validation.txt is already downloaded.
File data/test.txt is already downloaded.


### Load the Twitter Named Entity Recognition corpus

We will work with a corpus, which contains tweets with NE tags. Every line of a file contains a pair of a token (word/punctuation symbol) and a tag, separated by a whitespace. Different tweets are separated by an empty line.

The function *read_data* reads a corpus from the *file_path* and returns two lists: one with tokens and one with the corresponding tags. You need to complete this function by adding a code, which will replace a user's nickname to `<USR>` token and any URL to `<URL>` token. You could think that a URL and a nickname are just strings which start with *http://* or *https://* in case of URLs and a *@* symbol for nicknames.

In [2]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token
            if token.startswith("http"):
                token = "<URL>"
            elif token.startswith("@"):
                token = "<USR>"

            ######################################
            ######### YOUR CODE HERE #############
            ######################################
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

And now we can load three separate parts of the dataset:
 - *train* data for training the model;
 - *validation* data for evaluation and hyperparameters tuning;
 - *test* data for final evaluation of the model.

In [3]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

You should always understand what kind of data you deal with. For this purpose, you can print the data running the following cell:

In [4]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Now you need to implement the function *build_dict* which will return {token or tag}$\to${index} and vice versa. 

In [5]:
from collections import defaultdict

In [6]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    # At first, add special tokens (or tags) to the dictionaries.
    # The first special token must have index 0.
    
    # Mapping tok2idx should contain each token or tag only once. 
    # To do so, you should:
    # 1. extract unique tokens/tags from the tokens_or_tags variable, which is not
    #    occur in special_tokens (because they could have non-empty intersection)
    # 2. index them (for example, you can add them into the list idx2tok
    # 3. for each token/tag save the index into tok2idx).
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    merged_tokens_or_tags = set()
    for tokens in tokens_or_tags:
        merged_tokens_or_tags = merged_tokens_or_tags.union(tokens)

    special_tokens = list(set(special_tokens))
    idx2tok.extend(special_tokens)
    for token in merged_tokens_or_tags:
        if token not in special_tokens:
            idx2tok.append(token)
            
    for i in range(len(idx2tok)):
        tok2idx[idx2tok[i]] = i
    
    return tok2idx, idx2tok

After implementing the function *build_dict* you can make dictionaries for tokens and tags. Special tokens in our case will be:
 - `<UNK>` token for out of vocabulary tokens;
 - `<PAD>` token for padding sentence to the same length when we create batches of sentences.

In [7]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

The next additional functions will help you to create the mapping between tokens and ids for a sentence. 

In [8]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

### Prepare data

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

num_of_words = len(idx2token)
num_of_tags = len(idx2tag)

max_length = 0
for sentence in train_tokens + validation_tokens + test_tokens:
    if len(sentence) > max_length:
        max_length = len(sentence)

X_train = [words2idxs(sentence) for sentence in train_tokens]
X_train = pad_sequences(maxlen=max_length, sequences=X_train, padding="post", value=token2idx["PAD"])

y_train = [tags2idxs(tags) for tags in train_tags]
y_train = pad_sequences(maxlen=max_length, sequences=y_train, padding="post", value=tag2idx["O"])
y_train = [to_categorical(i, num_classes=num_of_tags) for i in y_train]

X_val = [words2idxs(sentence) for sentence in validation_tokens]
X_val = pad_sequences(maxlen=max_length, sequences=X_val, padding="post", value=token2idx["PAD"])

y_val = [tags2idxs(tags) for tags in validation_tags]
y_val = pad_sequences(maxlen=max_length, sequences=y_val, padding="post", value=tag2idx["O"])
y_val = [to_categorical(i, num_classes=num_of_tags) for i in y_val]

X_test = [words2idxs(sentence) for sentence in test_tokens]
X_test = pad_sequences(maxlen=max_length, sequences=X_test, padding="post", value=token2idx["PAD"])

y_test = [tags2idxs(tags) for tags in test_tags]
y_test = pad_sequences(maxlen=max_length, sequences=y_test, padding="post", value=tag2idx["O"])
y_test = [to_categorical(i, num_classes=num_of_tags) for i in y_test]

## Build a recurrent neural network

This is the most important part of the assignment. Here we will specify the network architecture based on TensorFlow building blocks. It's fun and easy as a lego constructor! We will create an LSTM network which will produce probability distribution over tags for each token in a sentence. To take into account both right and left contexts of the token, we will use Bi-Directional LSTM (Bi-LSTM). Dense layer will be used on top to perform tag classification.  

In [10]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, Dropout, Bidirectional
import numpy as np

In [11]:
inputs = Input((max_length,))
x = Embedding(input_dim=num_of_words, output_dim=300, input_length=max_length)(inputs)
x = Dropout(0.2)(x)
x = Bidirectional(LSTM(64, return_sequences=True, activation="tanh"))(x)
outputs = TimeDistributed(Dense(num_of_tags, activation="softmax"))(x)

model = Model(inputs, outputs)
model.summary()

Metal device set to: Apple M1 Max
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 41)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 41, 300)           6151200   
_________________________________________________________________
dropout (Dropout)            (None, 41, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 41, 128)           186880    
_________________________________________________________________
time_distributed (TimeDistri (None, 41, 21)            2709      
Total params: 6,340,789
Trainable params: 6,340,789
Non-trainable params: 0
_________________________________________________________________


2022-02-17 15:20:22.663087: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-17 15:20:22.663219: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, np.array(y_train), validation_data=(X_val, np.array(y_val)), batch_size=64, epochs=100)

2022-02-17 15:20:22.980225: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-17 15:20:22.980390: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2022-02-17 15:20:23.633156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 15:20:23.749831: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 15:20:23.769556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 15:20:23.919495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 15:20:23.936002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - ETA: 0s - loss: 0.5999 - accuracy: 0.9489

2022-02-17 15:20:28.571868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 15:20:28.612549: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 15:20:28.618958: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


91/91 [==============================] - 6s 54ms/step - loss: 0.5999 - accuracy: 0.9489 - val_loss: 0.1890 - val_accuracy: 0.9700
Epoch 2/100
91/91 [==============================] - 4s 47ms/step - loss: 0.1991 - accuracy: 0.9679 - val_loss: 0.1866 - val_accuracy: 0.9700
Epoch 3/100
91/91 [==============================] - 4s 46ms/step - loss: 0.1971 - accuracy: 0.9679 - val_loss: 0.1848 - val_accuracy: 0.9700
Epoch 4/100
91/91 [==============================] - 4s 46ms/step - loss: 0.1942 - accuracy: 0.9679 - val_loss: 0.1820 - val_accuracy: 0.9700
Epoch 5/100
91/91 [==============================] - 4s 48ms/step - loss: 0.1908 - accuracy: 0.9679 - val_loss: 0.1788 - val_accuracy: 0.9700
Epoch 6/100
91/91 [==============================] - 4s 46ms/step - loss: 0.1855 - accuracy: 0.9679 - val_loss: 0.1739 - val_accuracy: 0.9700
Epoch 7/100
91/91 [==============================] - 4s 46ms/step - loss: 0.1799 - accuracy: 0.9679 - val_loss: 0.1688 - val_accuracy: 0.9700
Epoch 8/100
91/91 

91/91 [==============================] - 4s 48ms/step - loss: 0.0330 - accuracy: 0.9900 - val_loss: 0.1508 - val_accuracy: 0.9758
Epoch 59/100
91/91 [==============================] - 4s 48ms/step - loss: 0.0320 - accuracy: 0.9904 - val_loss: 0.1567 - val_accuracy: 0.9764
Epoch 60/100
91/91 [==============================] - 4s 48ms/step - loss: 0.0318 - accuracy: 0.9904 - val_loss: 0.1582 - val_accuracy: 0.9761
Epoch 61/100
91/91 [==============================] - 4s 48ms/step - loss: 0.0313 - accuracy: 0.9905 - val_loss: 0.1599 - val_accuracy: 0.9758
Epoch 62/100
91/91 [==============================] - 4s 46ms/step - loss: 0.0293 - accuracy: 0.9910 - val_loss: 0.1607 - val_accuracy: 0.9762
Epoch 63/100
91/91 [==============================] - 4s 48ms/step - loss: 0.0286 - accuracy: 0.9913 - val_loss: 0.1584 - val_accuracy: 0.9755
Epoch 64/100
91/91 [==============================] - 4s 48ms/step - loss: 0.0280 - accuracy: 0.9913 - val_loss: 0.1620 - val_accuracy: 0.9758
Epoch 65/100

In [13]:
model.evaluate(X_test, np.array(y_test))

23/23 [==============================] - 1s 21ms/step - loss: 0.2908 - accuracy: 0.9732


[0.29075345396995544, 0.9731842279434204]